In [144]:
# Import Dependencies

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pylab import *
from config import password

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

import pymysql
pymysql.install_as_MySQLdb()

In [145]:
# Create references to the CSVs and import into Pandas DataFrames

measurements_csv = os.path.join("clean_hawaii_measurements.csv")
stations_csv = os.path.join("clean_hawaii_stations.csv")

measurements_df = pd.read_csv(measurements_csv, low_memory=False)
stations_df = pd.read_csv(stations_csv, low_memory=False)

In [146]:
# # Construct MySQL location
# location = 'mysql://root:' + password + '@localhost:3306/hawaii.sqlite'

# # Create a connection to DB using the MySQL Connect Engine
# engine = create_engine(location)
# conn = engine.connect()

In [147]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

In [148]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class

In [149]:
# engine.execute('SELECT * FROM hawaii LIMIT 5').fetchall()

In [150]:
# Check for db
!ls *.sqlite

hawaii.sqlite


In [151]:
# Check for tables
engine.table_names()

['stations']

inspector = inspect(engine)
columns = inspector.get_columns('hawaii')
for c in columns:
    print(c['name'], c["type"])

In [152]:
# Set an object to use `declarative_base` from SQLAlchemy to model tables as ORM classes
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

# Pass this class to convert classes to sql tables
# Make sure to specify types for each column, e.g. Integer, Text, etc.

class Stations(Base):
    __tablename__ = 'stations'

    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    

In [153]:
# Primary Key is defined
inspector.get_columns('stations')

[{'autoincrement': 'auto',
  'default': None,
  'name': 'station',
  'nullable': False,
  'primary_key': 1,
  'type': VARCHAR()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'name',
  'nullable': True,
  'primary_key': 0,
  'type': VARCHAR()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'latitude',
  'nullable': True,
  'primary_key': 0,
  'type': FLOAT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'longitude',
  'nullable': True,
  'primary_key': 0,
  'type': FLOAT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'elevation',
  'nullable': True,
  'primary_key': 0,
  'type': FLOAT()}]

In [154]:
# if using automap:
#Base.prepare()

In [155]:
# Use `create_all` to create table in the database
Base.metadata.create_all(engine)

In [156]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = stations_df.to_dict(orient='records')

In [157]:
print(data)

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}, {'station': 'USC00519523', 'name': 'WAIMANALO EXPERIMENTAL FARM, HI US', 'latitude': 21.33556, 'longitude': -157.71139, 'elevation': 19.5}, {'station': 'USC00519281', 'name': 'WAIHEE 837.5, HI US', 'latitude': 21.45167, 'longitude': -157.84888999999995, 'elevation': 32.9}, {'station': 'USC00511918', 'name': 'HONOLULU OBSERVATORY 702.2, HI US', 'latitude': 2

In [158]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [159]:
# Save the reference to the `stations` table as a variable called `table`
# Needed to bulk upload data, rather than add through individual objects
table = sqlalchemy.Table('stations', metadata, autoload=True)

In [160]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [161]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

engine.execute('SELECT * FROM stations LIMIT 5').fetchall()

In [164]:
results = session.query(Stations)

for row in results:
    print(row.__dict__)

In [165]:
first_result = session.query(Stations).first()
first_result.__dict__

AttributeError: 'NoneType' object has no attribute '__dict__'